In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

In [2]:
football_df = pd.read_csv("data.csv")

In [3]:
football_df.head()

,Unnamed: 0,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,...,lat/lng,type_of_shot,type_of_combined_shot,match_id,team_id,remaining_min.1,power_of_shot.1,knockout_match.1,remaining_sec.1,distance_of_shot.1
0,0,10.0,167.0,72.0,10.0,1.0,0.0,2000-01,27.0,38.0,...,"45.539131, -122.651648",shot - 30,NaN,20000012,1610612747,10.00,1.0,50.608,54.2000,38.0
1,1,12.0,-157.0,0.0,10.0,1.0,0.0,2000-01,22.0,35.0,...,"45.539131, -122.651648",shot - 45,NaN,20000012,1610612747,10.00,1.0,28.800,22.0000,35.0
2,2,35.0,-101.0,135.0,7.0,1.0,0.0,2000-01,45.0,36.0,...,"45.539131, -122.651648",shot - 25,NaN,20000012,1610612747,92.64,1.0,0.000,63.7216,54.4
3,3,43.0,138.0,175.0,6.0,1.0,0.0,2000-01,52.0,42.0,...,"45.539131, -122.651648",NaN,shot - 3,20000012,1610612747,NaN,1.0,122.608,52.0000,42.0
4,4,155.0,0.0,0.0,NaN,2.0,0.0,2000-01,19.0,20.0,...,"45.539131, -122.651648",NaN,shot - 1,20000012,1610612747,42.64,2.0,0.000,19.0000,20.0


In [4]:
football_df.columns

Index(['Unnamed: 0', 'match_event_id', 'location_x', 'location_y',
       'remaining_min', 'power_of_shot', 'knockout_match', 'game_season',
       'remaining_sec', 'distance_of_shot', 'is_goal', 'area_of_shot',
       'shot_basics', 'range_of_shot', 'team_name', 'date_of_game',
       'home/away', 'shot_id_number', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id', 'team_id', 'remaining_min.1',
       'power_of_shot.1', 'knockout_match.1', 'remaining_sec.1',
       'distance_of_shot.1'],
      dtype='object')

# Data Cleaning

In [5]:
unique_mins = football_df["remaining_min"].unique()
unique_mins = unique_mins[~np.isnan(unique_mins)]

count = football_df.groupby("remaining_min").size()
count /= count.sum()

In [8]:
unique_mins

array([10.,  7.,  6.,  9.,  8.,  3.,  1., 11.,  0.,  5.,  2.,  4.])

In [716]:
football_df["remaining_min"] = football_df.remaining_min.map(lambda x: np.random.choice(unique_mins, p=count) if np.isnan(x) else x)

In [718]:
#Dropping unecessary and redundant columns

In [717]:
columns_to_drop = [
    "team_name",
    "home/away",
    "date_of_game",
    "team_id",
    "remaining_min.1",
    "power_of_shot.1",
    "knockout_match.1",
    "remaining_sec.1",
    "distance_of_shot.1",
    "Unnamed: 0",
]

In [719]:
football_df.drop(columns_to_drop, axis=1, inplace=True)

In [720]:
football_df.columns

Index(['match_event_id', 'location_x', 'location_y', 'remaining_min',
       'power_of_shot', 'knockout_match', 'game_season', 'remaining_sec',
       'distance_of_shot', 'is_goal', 'area_of_shot', 'shot_basics',
       'range_of_shot', 'shot_id_number', 'lat/lng', 'type_of_shot',
       'type_of_combined_shot', 'match_id'],
      dtype='object')

In [721]:
football_df.match_event_id.fillna(value = -1, inplace = True)

In [722]:
unique_pos = football_df["power_of_shot"].unique()
unique_pos = unique_pos[~np.isnan(unique_pos)]

count_pos = football_df.groupby("power_of_shot").size()
count_pos /= count_pos.sum()

In [723]:
football_df["power_of_shot"] = football_df.power_of_shot.map(lambda x: np.random.choice(unique_pos, p = count_pos) if np.isnan(x) else x )

In [724]:
count_knockout = football_df.groupby("knockout_match").size()
count_knockout /= count_knockout.sum()

In [725]:
football_df["knockout_match"] = football_df.knockout_match.map(lambda x: np.random.choice([0,1], p = count_knockout) if np.isnan(x) else x )

In [726]:
# football_df.drop(["type_of_shot","type_of_combined_shot"], axis = 1, inplace=True)

In [727]:
football_df[["lat","lng"]] = football_df["lat/lng"].str.split(",",expand = True)

In [728]:
football_df.drop("lat/lng", inplace=True, axis = 1)

In [729]:
football_df["game_season"] = football_df["game_season"].fillna(method = 'backfill')

In [730]:
football_df["range_of_shot"] = football_df["range_of_shot"].astype('category')
football_df["range_of_shot"] = football_df["range_of_shot"].cat.codes

football_df["shot_basics"] = football_df["shot_basics"].astype('category')
football_df["shot_basics"] = football_df["shot_basics"].cat.codes

football_df["area_of_shot"] = football_df["area_of_shot"].astype('category')
football_df["area_of_shot"] = football_df["area_of_shot"].cat.codes

football_df["game_season"] = football_df["game_season"].astype('category')
football_df["game_season"] = football_df["game_season"].cat.codes

In [731]:
football_df["shot_id_number"] = football_df.index
football_df.shot_id_number = football_df.shot_id_number.map(lambda x: x+1)

In [732]:
football_df.type_of_shot.fillna(football_df.type_of_combined_shot, inplace=True)
football_df.drop("type_of_combined_shot", axis = 1, inplace=True)

In [733]:
football_df["type_of_shot"] = football_df.type_of_shot.map(lambda x: int(str(x).split(" - ")[1]), na_action="ignore")

In [734]:
unique_types_of_shot = football_df["type_of_shot"].unique()
unique_types_of_shot = unique_types_of_shot[~np.isnan(unique_types_of_shot)]

count_types_of_shot = football_df.groupby("type_of_shot").size()
count_types_of_shot /= count_types_of_shot.sum()

In [735]:
football_df["type_of_shot"] = football_df.type_of_shot.map(lambda x: np.random.choice(unique_types_of_shot,p=count_types_of_shot) if np.isnan(x) else x)

In [736]:
imputer = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)
imputer = imputer.fit(football_df.iloc[:,6:7])
football_df.iloc[:,6:7] = imputer.transform(football_df.iloc[:,6:7])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [737]:
imputer = Imputer(missing_values = 'NaN', strategy = 'mean', axis = 0)
imputer = imputer.fit(football_df.iloc[:,1:3])
football_df.iloc[:,1:3] = imputer.transform(football_df.iloc[:,1:3])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [738]:
imputer = Imputer(missing_values = 'NaN', strategy = 'median', axis = 0)
imputer = imputer.fit(football_df.iloc[:,15:])
football_df.iloc[:,15:] = imputer.transform(football_df.iloc[:,15:])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [739]:
imputer = Imputer(missing_values = 'NaN', strategy = 'most_frequent', axis = 0)
imputer = imputer.fit(football_df.iloc[:,7:8])
football_df.iloc[:,7:8] = imputer.transform(football_df.iloc[:,7:8])

/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:66: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [768]:
unique_dist_of_shot = football_df["distance_of_shot"].unique()
unique_dist_of_shot = unique_dist_of_shot[~np.isnan(unique_dist_of_shot)]

count_dist_of_shot = football_df.groupby("distance_of_shot").size()
count_dist_of_shot /= count_dist_of_shot.sum()

In [769]:
football_df["distance_of_shot"] = football_df.distance_of_shot.map(lambda x: np.random.choice(unique_types_of_shot,p=count_types_of_shot) if np.isnan(x) else x)

In [770]:
football_df.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,shot_id_number,type_of_shot,match_id,lat,lng
0,10.0,167.0,72.0,10.0,1.0,0.0,4.0,27.0,38.0,NaN,5,4,0,1,30,20000012.0,45.539131,-122.651648
1,12.0,-157.0,0.0,10.0,1.0,0.0,4.0,22.0,35.0,0.0,2,4,2,2,45,20000012.0,45.539131,-122.651648
2,35.0,-101.0,135.0,7.0,1.0,0.0,4.0,45.0,36.0,1.0,1,4,0,3,25,20000012.0,45.539131,-122.651648
3,43.0,138.0,175.0,6.0,1.0,0.0,4.0,52.0,42.0,0.0,4,4,0,4,3,20000012.0,45.539131,-122.651648
4,155.0,0.0,0.0,4.0,2.0,0.0,4.0,19.0,20.0,1.0,0,0,4,5,1,20000012.0,45.539131,-122.651648


In [771]:
football_df.isnull().sum()

match_event_id         0
location_x             0
location_y             0
remaining_min          0
power_of_shot          0
knockout_match         0
game_season            0
remaining_sec          0
distance_of_shot       0
is_goal             6268
area_of_shot           0
shot_basics            0
range_of_shot          0
shot_id_number         0
type_of_shot           0
match_id               0
lat                    0
lng                    0
dtype: int64

# Splitting data into Training and Testing data

In [772]:
condition = np.isnan(football_df["is_goal"])

In [773]:
unlabelled_data = football_df[condition]

In [774]:
unlabelled_data.shape

(6268, 18)

In [775]:
labelled_data = football_df[~condition]

In [776]:
labelled_data.shape

(24429, 18)

In [777]:
labelled_data = shuffle(labelled_data)
labelled_data.head()

,match_event_id,location_x,location_y,remaining_min,power_of_shot,knockout_match,game_season,remaining_sec,distance_of_shot,is_goal,area_of_shot,shot_basics,range_of_shot,shot_id_number,type_of_shot,match_id,lat,lng
22684,498.0,-237.000000,-24.0,4.0,4.0,0.0,19.0,1.0,43.0,0.0,2,2,1,22685,17,21501065.0,33.552026,-112.071667
19162,480.0,7.383876,47.0,3.0,4.0,0.0,15.0,3.0,28.0,0.0,2,1,2,19163,38,21100679.0,29.740325,-95.365762
11608,450.0,71.000000,209.0,9.0,4.0,0.0,11.0,57.0,42.0,0.0,4,4,-1,11609,11,20700113.0,42.982923,-71.446094
7667,391.0,0.000000,0.0,4.0,3.0,0.0,9.0,24.0,20.0,0.0,0,0,-1,7668,39,20500064.0,46.667324,-94.419250
30053,470.0,27.000000,259.0,0.0,4.0,1.0,0.0,0.0,46.0,0.0,0,5,1,30054,8,49600041.0,42.982923,-71.446094


In [778]:
Y = labelled_data["is_goal"]

In [779]:
Y.shape

(24429,)

In [780]:
X = labelled_data.drop(["is_goal","shot_id_number"], axis = 1)

In [781]:
X.shape

(24429, 16)

In [782]:
shot_id_number = unlabelled_data.shot_id_number

In [783]:
X_pred = unlabelled_data.drop(["is_goal","shot_id_number"], axis = 1)

In [784]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, random_state = 0)

In [785]:
from sklearn.ensemble import RandomForestClassifier

In [786]:
RF = RandomForestClassifier(n_estimators = 500)

In [787]:
RF.fit(X_train,Y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=500,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [788]:
RF.score(X_test,Y_test)

0.6123618501841998

In [789]:
Y_pred = RF.predict(X_pred)

In [790]:
d = {
    "shot_id_number" : shot_id_number,
    "is_goal" : Y_pred
}

In [791]:
final = pd.DataFrame(d)

In [792]:
final.set_index('shot_id_number', inplace=True)

In [793]:
final

,is_goal
shot_id_number,
1,0.0
8,0.0
17,1.0
20,1.0
22,0.0
33,0.0
34,0.0
35,1.0
36,1.0


In [794]:
final.to_csv("bipin_kalra_110597_code_1.csv")